# Improved SCA based on SVM
---

This project implements the side-channel attack described in the paper *"An Improved Side-Channel Attack Based on Support Vector Machine" by Zeng et al.* \
The paper frames power analysis of AES as a classification problem, leveraging classifiers to break the encryption.

The following notebooks demonstrate breaking a single byte of a basic AES encryption key.

---

In this notebook, we will set up the hardware and capture power traces. \
If you don't have a Chipwhisperer Nano you can skip this part and use the included traces.

---
#### 1. Connecting the device, building and flashing the firmware

In [2]:
import chipwhisperer as cw

In [3]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
CRYPTO_TARGET='TINYAES128C'
SS_VER='SS_VER_2_1'

In [4]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [5]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
Blank crypto options, building for AES128
Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
Blank crypto options, building for AES128
make[1]: '.dep' is up to date.
Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
Blank crypto options, building for AES128
arm-none-eabi-gcc (GNU Arm Embedded Toolchain 10-2020-q4-major) 10.2.1 20201103 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Welcome to another exciting ChipWhisperer target build!!
.
Assembling: .././hal/stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -ffunction-sections -DF_C

`Note:` Make sure to remove or comment out the *ADD_JITTER* in the firmware.

In [6]:
cw.program_target(scope, prog, "../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F03xx4/03xx6
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5859 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5859 bytes


#### 2. Setting the encryption key and capturing the power traces

In [7]:
key = bytes.fromhex("2b7e151628aed2a6abf7158809cf4f3c")
target.set_key(key)
print(f"Key set to: {key.hex()}")

Key set to: 2b7e151628aed2a6abf7158809cf4f3c


In [8]:
import numpy as np
from tqdm.notebook import trange

traces = []
textins = []
textouts = []

# Number of traces to capture
N = 20000

ktp = cw.ktp.Basic()

# Capture traces in a loop with a progress bar.
for i in trange(N, desc='Capturing traces'):
    scope.arm()  # Arm the scope.
    
    # Get the next key and plaintext for the following iteration.
    _, text = ktp.next()
    
    # Send the current plaintext to the target.
    target.simpleserial_write('p', text)
    
    # Capture the power trace.
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue  # Skip this trace if capture fails.
    
    # Read the target's response.
    response = target.simpleserial_read('r', 16)
    
    # Save the captured trace, plaintext, and key.
    traces.append(scope.get_last_trace())
    textins.append(text)
    textouts.append(response)

# Convert the list of traces to a NumPy array for easier processing.
traces = np.array(traces)
print(f"Captured {len(traces)} traces.")

Capturing traces:   0%|          | 0/20000 [00:00<?, ?it/s]

Captured 20000 traces.


#### 3. Saving the traces and disconnecting the hardware

In [12]:
import os

project_folder = 'SCA_SVM_DATA'    
traces_folder = f'{project_folder}/traces'
os.makedirs(traces_folder, exist_ok = True)    
project_file = f'{traces_folder}/training_traces.cwp'
project = cw.create_project(project_file, overwrite = True)
for i, (trace, textin, textout) in enumerate(zip(traces, textins, textouts)):
    project.traces.append(cw.Trace(wave=trace, textin=textin, textout=textout, key=key))
project.save()

print(f"{len(traces)} traces saved to {project_file}")

20000 traces saved to SCA_SVM_DATA/traces/training_traces.cwp


In [13]:
scope.dis()
target.dis()

### That's it! We're ready to train the models